In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from sklearn import metrics

import cPickle as pickle


In [3]:
# Load Data
joined = None
cleaned = None
cvectr = None
nmf = None
text_nmf = None
components_df = None

    
with open('WranglingPickles/cleaned.pkl', 'rb') as f:
    cleaned = pickle.load(f)
    
with open('WranglingPickles/cvectr.pkl', 'rb') as f:
    cvectr = pickle.load(f)
    
with open('WranglingPickles/nmf.pkl', 'rb') as f:
    nmf = pickle.load(f)
    
with open('WranglingPickles/text_nmf.pkl', 'rb') as f:
    text_nmf = pickle.load(f)
    
with open('WranglingPickles/components_df.pkl', 'rb') as f:
    components_df = pickle.load(f)
    
with open('WranglingPickles/Joined.pkl','rb') as f:
    joined = pickle.load(f)

In [13]:
cols = ['Coding_Latex', "Genetics", 'Psychiatric_Disorder', 'Attention', 'ImmuneSystem_Cancer',
        'Neurons', 'Animal_Experiments', 'Alzheimers_Dementia_Parkinson',  'Sleep',
        "Dependency", "Pain_MotorFunction", 'Stroke_Aneurysm_Damage', 'Developmental_Disorders', 
        'Brain_Mapping', 'Memory']

text_nmf['ID'] = list(map(lambda j: j[0], joined))
text_nmf['Journal'] = list(map(lambda j: j[1], joined))
text_nmf['Date'] = list(map(lambda j: j[2], joined))

text_nmf.columns = cols +['ID', 'Journal', 'Date']


floatCols = text_nmf.columns[text_nmf.dtypes == 'float64']




text_nmf['Max_Topic_Name'] = text_nmf[floatCols].apply(lambda row: row.sort_values(ascending = False)[:1].index[0], axis = 1)
text_nmf['Max_Topic_Val'] = text_nmf[floatCols].apply(lambda row: row.sort_values(ascending = False)[0], axis = 1)

text_nmf.head()



,Coding_Latex,Genetics,Psychiatric_Disorder,Attention,ImmuneSystem_Cancer,Neurons,Animal_Experiments,Alzheimers_Dementia_Parkinson,Sleep,Dependency,Pain_MotorFunction,Stroke_Aneurysm_Damage,Developmental_Disorders,Brain_Mapping,Memory,ID,Journal,Date,Max_Topic_Name,Max_Topic_Val
0,0.001273,0.000000,0.247913,0.000000,0.199396,0.471529,0.167966,0.060914,0.017921,2.139161,0.000000,0.000000,0.961869,0.186803,0.538371,581de078b067bc05a36409c1,Neuroscience,2014_Dec,Dependency,2.139161
1,0.004978,0.000000,0.000000,0.317037,0.136185,0.468468,0.206836,0.000000,0.084690,0.496419,0.512749,0.000000,0.000000,0.064152,0.000000,581de078b067bc05a36409c2,Neuroscience,2010_Oct,Pain_MotorFunction,0.512749
2,0.033887,0.043568,0.000000,0.722161,0.448714,0.774374,0.223858,0.000000,0.000000,0.000000,0.502211,0.000000,0.000000,0.027228,0.000000,581de078b067bc05a36409c3,Neuroscience,2012_Jul,Neurons,0.774374
3,0.000000,0.000000,0.000000,1.271206,0.000000,0.000000,0.000000,0.010501,0.000000,0.000000,0.000000,0.006899,0.062925,0.821910,0.798151,581de078b067bc05a36409c4,Neuroscience,2016_Aug,Attention,1.271206
4,0.000000,0.000000,0.024419,0.000000,0.153504,2.570706,0.249871,0.000000,0.000000,0.083720,0.515074,0.000000,0.013573,0.000000,0.000000,581de078b067bc05a36409c5,Neuroscience,2016_Jun,Neurons,2.570706


In [31]:
# Recommender will calculate cosine_similarity between articles of same max_topic and order them
rand_ix = np.random.choice(text_nmf.index)
rand_article = text_nmf.ix[rand_ix]
print rand_ix

topic = rand_article.Max_Topic_Name


data = text_nmf[text_nmf.Max_Topic_Name == topic]
print data.shape
sims = pd.DataFrame(cosine_similarity(data[floatCols]), index= data.index, columns = data.index)
recs = sims.ix[rand_ix].sort_values(ascending = False)[1:11]

14845
(2956, 20)


In [87]:

def Recommend(article):
    topic = article.Max_Topic_Name
    data = text_nmf[text_nmf.Max_Topic_Name == topic]
    sims = pd.Series(cosine_similarity(article[floatCols].reshape(1,-1), data[floatCols])[0], index = data.index)
    recs = sims.sort_values(ascending=False)
    return recs[recs<1.][:10]

In [88]:
Recommend(rand_article)

10022    0.996317
8506     0.991925
9962     0.986794
14357    0.984801
5576     0.982586
5562     0.981192
7092     0.980721
24836    0.980543
14503    0.979785
8411     0.979215
dtype: float64